In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
data=pd.read_csv('../data/cleaned_hm.csv')

In [7]:
data=data[data['reflection_period']=='24h']

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49831 entries, 0 to 100534
Data columns (total 9 columns):
hmid                     49831 non-null int64
wid                      49831 non-null int64
reflection_period        49831 non-null object
original_hm              49831 non-null object
cleaned_hm               49831 non-null object
modified                 49831 non-null bool
num_sentence             49831 non-null int64
ground_truth_category    6818 non-null object
predicted_category       49831 non-null object
dtypes: bool(1), int64(3), object(5)
memory usage: 3.5+ MB


In [9]:
import nltk
from nltk.corpus import brown
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [10]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
nltk.download("stopwords")
nltk.download("wordnet")
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chenxishi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chenxishi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    number_free = "".join(ch for ch in punc_free if not ch.isdigit())
    normalized = " ".join(lemma.lemmatize(word) for word in number_free.split())
    normalized_v = " ".join(lemma.lemmatize(word, "v") for word in normalized.split())
    return normalized_v

In [12]:
clean('went to schools')

'go school'

In [13]:
data['stem_hm'] = [clean(senten) for senten in data["cleaned_hm"].tolist()]

In [14]:
content=list(map(lambda x:x.split(),data['stem_hm']))

In [15]:
from gensim import corpora, models
import gensim

In [16]:
dictionary = corpora.Dictionary(content)
corpus = [dictionary.doc2bow(text) for text in content]

In [17]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=15, id2word = dictionary, passes=20)


In [18]:
ldamodel.save('../output/lda20pass15topic_with24.model') 

In [19]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
visualization = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/chenxishi/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [20]:
visualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.062732 -0.102384       1        1  11.349138
2     -0.102996 -0.109971       2        1   8.508636
0      0.010069 -0.132118       3        1   8.185329
14    -0.054411 -0.043249       4        1   7.058591
6     -0.055350  0.127394       5        1   6.972802
4     -0.128741 -0.141437       6        1   6.579214
9      0.044687 -0.183556       7        1   6.447265
11    -0.073723  0.058574       8        1   6.355464
1     -0.065396  0.084720       9        1   6.270316
5     -0.155875  0.032534      10        1   6.142571
10     0.264841 -0.112564      11        1   6.035408
13    -0.063288  0.199815      12        1   5.909357
7      0.206612  0.206226      13        1   5.041174
8      0.273831 -0.008334      14        1   4.913836
3     -0.037529  0.124349      15        1   4.230899, topic_info=     Category          Freq         Term         Total  loglift  logprob
term                                                                    
10    Default   8955.000000        happy   8955.000000  30.0000  30.0000
144   Default   5353.000000         time   5353.000000  29.0000  29.0000
36    Default   8123.000000         make   8123.000000  28.0000  28.0000
264   Default   5641.000000         work   5641.000000  27.0000  27.0000
18    Default   5044.000000       friend   5044.000000  26.0000  26.0000
37    Default   4198.000000          new   4198.000000  25.0000  25.0000
55    Default   2927.000000        watch   2927.000000  24.0000  24.0000
31    Default   3214.000000         last   3214.000000  23.0000  23.0000
33    Default   2657.000000        night   2657.000000  22.0000  22.0000
88    Default   2412.000000         take   2412.000000  21.0000  21.0000
250   Default   2282.000000       dinner   2282.000000  20.0000  20.0000
408   Default   2000.000000       moment   2000.000000  19.0000  19.0000
189   Default   2347.000000         home   2347.000000  18.0000  18.0000
503   Default   1806.000000         hour   1806.000000  17.0000  17.0000
19    Default   3408.000000         good   3408.000000  16.0000  16.0000
279   Default   2064.000000          eat   2064.000000  15.0000  15.0000
174   Default   1880.000000         give   1880.000000  14.0000  14.0000
334   Default   1937.000000         life   1937.000000  13.0000  13.0000
312   Default   1823.000000          buy   1823.000000  12.0000  12.0000
3     Default   7640.000000           go   7640.000000  11.0000  11.0000
106   Default   5076.000000          day   5076.000000  10.0000  10.0000
104   Default   2339.000000         play   2339.000000   9.0000   9.0000
178   Default   2022.000000        first   2022.000000   8.0000   8.0000
35    Default   2586.000000         come   2586.000000   7.0000   7.0000
9     Default  10037.000000          get  10037.000000   6.0000   6.0000
381   Default   1491.000000          dog   1491.000000   5.0000   5.0000
1402  Default   1279.000000        event   1279.000000   4.0000   4.0000
58    Default   1766.000000         game   1766.000000   3.0000   3.0000
534   Default   2511.000000         find   2511.000000   2.0000   2.0000
582   Default   1525.000000          old   1525.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
2198  Topic15    150.813065         name    151.820190   3.1561  -4.8141
1107  Topic15    148.969589   unexpected    149.976715   3.1560  -4.8264
1888  Topic15    148.078262          fry    149.085388   3.1560  -4.8324
967   Topic15    135.409943         pack    136.417068   3.1553  -4.9219
1345  Topic15    128.655029     youngest    129.662155   3.1550  -4.9730
1847  Topic15    128.050308        grill    129.057434   3.1549  -4.9777
581   Topic15    126.612259         star    127.619377   3.1548  -4.9890
2237  Topic15    121.861710  significant    122.868828   3.1545  -5.0273
142   Topic15    121.535950  grandmother    12